In [ ]:
! pip install torchvision
!  pip install torchsummary
! pip install torch
!pip install Pillow==5.3.0

    100% |████████████████████████████████| 2.0MB 10.9MB/s 
  Found existing installation: Pillow 5.4.1
    Uninstalling Pillow-5.4.1:
      Successfully uninstalled Pillow-5.4.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DATA_DIR="drive/My Drive/iscd/train"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import sys
sys.argv=['--dataset','clf','--dataroot','drive/My Drive/iscd/train','--worker','2','--batchSize','32','--imageSize','64',
          '--nz','100','--ngf','64','--ndf','64','--niter','11','--lr','.0002','--beta1','0.5','--ngpu','1'
         ]
parser  = argparse.ArgumentParser()
parser.add_argument("input_file",help="Input image, directory, or npy.")
parser.add_argument('--dataset',required=False,default='clf',help='clf')
parser.add_argument('--dataroot',required=False,help='path to dataset')
parser.add_argument('--worker',type=int,help='number of dataloading worker',default=2)
parser.add_argument('--batchSize',type=int,default=32,help='input batch size')
parser.add_argument('--imageSize',type=int,default=64,help='Size of the latent z vector')
parser.add_argument('--nz',type=int,default=100)
parser.add_argument('--ngf',type=int,default=64)
parser.add_argument('--ndf',type=int,default=64)
parser.add_argument('--niter',type=int,default=11,help='')
parser.add_argument('--lr',type=float,default=0.0002,help='learning rate defult=0.0002')
parser.add_argument('--beta1',type=float,default=0.5,help='number of gups to use')
parser.add_argument('--cuda',action='store_true',help='enables cuda')
parser.add_argument('--ngpu',type=int,default=1,help='enable cuda')
parser.add_argument('--netG',default='',help='path to netG')
parser.add_argument('--netD',default='',help='path to netD')
parser.add_argument('--outf',default='.',help='folder to output image and model check point')
parser.add_argument('--manualSeed',type=int,help='manual seed')

args = parser.parse_args()
print(args)


Namespace(batchSize=32, beta1=0.5, cuda=False, dataroot='drive/My Drive/iscd/train', dataset='clf', imageSize=64, input_file='clf', lr=0.0002, manualSeed=None, ndf=64, netD='', netG='', ngf=64, ngpu=1, niter=11, nz=100, outf='.', worker=2)


In [ ]:
try:
  os.makedirs(args.outf)
except OSError:
  pass
if args.manualSeed is None:
  args.manualSeed = random.randint(1,10000)
print("Random Seed :",args.manualSeed)
random.seed(args.manualSeed)
torch.manual_seed(args.manualSeed)
if args.cuda:
  torch.cuda.manual_seed_all(args.manualSeed)
cudnn.benchmark=True
if torch.cuda.is_available() and not args.cuda:
  print("Warning You have a CUDA device so should probably run with --cuda")
if args.dataset in ['clf']:
    # Foldr dataset
    dataset = dset.ImageFolder(root=args.dataroot,
                               transform=transforms.Compose([
                                   transforms.Resize(args.imageSize),
                                   transforms.CenterCrop(args.imageSize),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               ]))
dataloader = torch.utils.data.DataLoader(dataset,batch_size=args.batchSize,
                                         shuffle=True,num_workers=int(args.worker))
ngpu = int(args.ngpu)
nz  = int(args.nz)
ngf  = int(args.ngf)
ndf  = int(args.ndf)
nc  = 3
def weight_init(m):
    classname=m.__class__.__name__
    if classname.find('Conv')!=-1:
      m.weight.data.normal_(0.0,0.02)
    elif classname.find('BatchNorm')!=-1:
      m.weight.data.normal_(1.0,0.02)
      m.bias.data.fill_(0)
class _netG(nn.Module):
  def __init__(self,ngpu):
    super(_netG,self).__init__()
    self.ngpu = ngpu
    self.main = nn.Sequential(
        #input is z going into a convoltuion into a convolution
        nn.ConvTranspose2d(nz,ngf*8,4,1,0,bias=False),
        nn.BatchNorm2d(ngf*8),
        nn.ReLU(True),
        nn.ConvTranspose2d(ngf*8,ngf*4,4,2,1,bias=False),
        nn.BatchNorm2d(ngf*4),
        nn.ReLU(True),
        nn.ConvTranspose2d(ngf*4,ngf*2,4,2,1,bias=False),
        nn.BatchNorm2d(ngf*2),
        nn.ReLU(True),
        nn.ConvTranspose2d(ngf*2,ngf,4,2,1,bias=False),
        nn.BatchNorm2d(ngf),
        nn.ReLU(True),
        nn.ConvTranspose2d(ngf,nc,4,2,1,bias=False),
        nn.Tanh()
    )
  def forward(self,input):
      output  =self.main(input)
      return output
netG = _netG(ngpu)
netG.apply(weight_init)
if(args.netG!=''):
  netG.load_state_dict(torch.load(args.netG))
print(netG)
class _netD(nn.Module):
  def __init__(self,ngpu):
    super(_netD,self).__init__()
    self.ngpu = ngpu
    self.main = nn.Sequential(
         nn.Conv2d(nc,ndf,4,2,1,bias=False),
         nn.LeakyReLU(0.2,inplace=True),
         nn.Conv2d(ndf,ndf*2,4,2,1,bias=False),
         nn.BatchNorm2d(ndf*2),
         nn.LeakyReLU(0.2,inplace=True),
         nn.Conv2d(ndf*2,ndf*4,4,2,1,bias=False),
         nn.BatchNorm2d(ndf*4),
         nn.LeakyReLU(0.2,inplace=True),
         nn.Conv2d(ndf*4,ndf*8,4,2,1,bias=False),
         nn.BatchNorm2d(ndf*8),
         nn.LeakyReLU(0.2,inplace=True),
         nn.Conv2d(ndf*8,1,4,1,0,bias=False),
         nn.Sigmoid()
    )
  def forward(self,input):
      output  =self.main(input)
      return output.view(-1,1).squeeze(1)
netD = _netD(ngpu)
netD.apply(weight_init)
if args.netD!='':
  netD.load_state_dict(troch.load(args.netD))
print(netD)

os.listdir('drive/My Drive/iscd/train')



Random Seed : 4201
_netG(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)
_netD

['00']

In [ ]:
criterion = nn.BCELoss()
input = torch.FloatTensor(args.batchSize, 3, args.imageSize, args.imageSize)
noise = torch.FloatTensor(args.batchSize, nz, 1, 1)
fixed_noise = torch.FloatTensor(args.batchSize, nz, 1, 1).normal_(0, 1)
label = torch.FloatTensor(args.batchSize)
real_label = 1
fake_label = 0

if args.cuda:
    netD.cuda()
    netG.cuda()
    criterion.cuda()
    input, label = input.cuda(), label.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

fixed_noise = Variable(fixed_noise)

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=args.lr, betas=(args.beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=args.lr, betas=(args.beta1, 0.999))

for epoch in range(args.niter):
    for i,data in enumerate(dataloader):
        print("i",i)
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu, _ = data
        batch_size = real_cpu.size(0)
        if args.cuda:
            real_cpu = real_cpu.cuda()
        input.resize_as_(real_cpu).copy_(real_cpu)
        label.resize_(batch_size).fill_(real_label)
        inputv = Variable(input)
        labelv = Variable(label)

        output = netD(inputv)
        errD_real = criterion(output, labelv)
        errD_real.backward()
        D_x = output.data.mean()

        # train with fake
        noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        labelv = Variable(label.fill_(fake_label))
        output = netD(fake.detach())
        errD_fake = criterion(output, labelv)
        errD_fake.backward()
        D_G_z1 = output.data.mean()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        labelv = Variable(label.fill_(real_label))  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, labelv)
        errG.backward()
        D_G_z2 = output.data.mean()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, args.niter, i, len(dataloader),
                 errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        if i % 10 == 0:
            vutils.save_image(real_cpu,
                    '%s/real_samples.png' % args.outf,
                    normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.data,
                    '%s/fake_samples_epoch_%03d.png' % (args.outf, epoch),
                    normalize=True)

    # do checkpointing
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (args.outf, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (args.outf, epoch))



i 0
[0/11][0/13] Loss_D: 2.8755 Loss_G: 1.8466 D(x): 0.1210 D(G(z)): 0.3381 / 0.1758
i 1
[0/11][1/13] Loss_D: 2.3269 Loss_G: 3.9216 D(x): 0.9998 D(G(z)): 0.8496 / 0.0301
i 2
[0/11][2/13] Loss_D: 0.7363 Loss_G: 5.8397 D(x): 1.0000 D(G(z)): 0.4596 / 0.0039
i 3
[0/11][3/13] Loss_D: 0.2091 Loss_G: 6.0256 D(x): 0.9999 D(G(z)): 0.1790 / 0.0032
i 4
[0/11][4/13] Loss_D: 0.1365 Loss_G: 5.5443 D(x): 0.9995 D(G(z)): 0.1206 / 0.0051
i 5
[0/11][5/13] Loss_D: 0.2743 Loss_G: 5.8510 D(x): 0.9853 D(G(z)): 0.1996 / 0.0034
i 6
[0/11][6/13] Loss_D: 0.1713 Loss_G: 6.2285 D(x): 0.9973 D(G(z)): 0.1422 / 0.0027
i 7
[0/11][7/13] Loss_D: 0.1719 Loss_G: 6.4419 D(x): 0.9925 D(G(z)): 0.1333 / 0.0020
i 8
[0/11][8/13] Loss_D: 0.0917 Loss_G: 6.4129 D(x): 0.9974 D(G(z)): 0.0813 / 0.0021
i 9
[0/11][9/13] Loss_D: 0.2166 Loss_G: 6.8835 D(x): 0.9710 D(G(z)): 0.1503 / 0.0014
i 10
[0/11][10/13] Loss_D: 0.1168 Loss_G: 6.7729 D(x): 0.9773 D(G(z)): 0.0759 / 0.0017
i 11
[0/11][11/13] Loss_D: 0.0674 Loss_G: 6.5762 D(x): 0.9907 D